In [197]:
# 修正版：star_ratingを数値に変換し、無効な値を除外
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/dangararara/lecture/miraisouzou/suku/odaiba_reviews_3.csv')

# star_ratingを数値に変換
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# NaNの値を持つ行を除外
df_clean = df.dropna(subset=['star_rating'])

print(f"元のデータ行数: {len(df)}")
print(f"クリーニング後のデータ行数: {len(df_clean)}")
print(f"除外された行数: {len(df) - len(df_clean)}")

# 店舗ごとの平均星評価を計算
shop_df = df_clean.groupby(['shop_name', 'shop_url'], as_index=False).agg({'star_rating': 'mean'})
recs_star = shop_df.sort_values('star_rating', ascending=False).head(5)

print("\n星評価上位5店舗:")
print(recs_star)

元のデータ行数: 21642
クリーニング後のデータ行数: 21638
除外された行数: 4

星評価上位5店舗:
           shop_name                                           shop_url  \
168       三代目 博多 だるま  https://tabelog.com/tokyo/A1313/A131306/13271860/   
202          日本料理　時宜  https://tabelog.com/tokyo/A1313/A131306/13048167/   
22   RISTORANTE OZIO  https://tabelog.com/tokyo/A1313/A131306/13055614/   
44    しゃぶしゃぶ 吟半 お台場店  https://tabelog.com/tokyo/A1313/A131306/13284306/   
235           鉄板焼 銀杏  https://tabelog.com/tokyo/A1313/A131306/13004858/   

     star_rating  
168         3.67  
202         3.62  
22          3.61  
44          3.58  
235         3.57  


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
reviews = df.groupby('shop_name')['review_text'].apply(lambda x: ' '.join(x)).reset_index()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews['review_text'])
user_query = ["ピザが美味しいレストラン"]
user_vec = vectorizer.transform(user_query)
similarities = cosine_similarity(user_vec, X)[0]
reviews['similarity'] = similarities
recs_text = reviews.sort_values('similarity', ascending=False).head(5)
